# Momentum Strategies

# 2. Investigating Momentum

## Data

In this section, we empirically investigate some concerns regarding AQR's new momentum product. On Canvas, find the data file, `data/momentum_data.xlsx`.

* The first tab contains the momentum factor as an excess return: $r^{\text{mom}}$.
* The second tab contains returns on portfolios corresponding to scored momentum deciles.
    * $r^{\text{mom}(1)}$ denotes the portfolio of stocks in the lowest momentum decile, the "losers" with the lowest past returns.
    * $r^{\text{mom}(10)}$ denotes the portfolio of stocks in the highest momentum decile.
* The third tab gives portfolios sorted by momentum and size.
    * $r^{\text{momsu}}$ denotes the portfolio of small stocks in the top 3 deciles of momentum scores.
    * $r^{\text{momBD}}$ denotes the portfolio of big-stocks in the bottom 3 deciles of momentum scores.

Note that the Fama-French momentum return, $\tilde{r}^{\text{mom,FF}}$, given in the first tab, is constructed by FF as,

$$\tilde{r}^{\text{mom,FF}} = \frac{1}{2}(r^{\text{momBU}} + r^{\text{momSU}}) - \frac{1}{2}(r^{\text{momBD}} + r^{\text{momSD}})$$

The Fama-French momentum return

$$\tilde{r}^{mom,FF} = \frac{1}{2}(\tilde{r}^{momBU} + \tilde{r}^{momSU}) - \frac{1}{2}(\tilde{r}^{momBD} + \tilde{r}^{momSD})$$

In [10]:
import pandas as pd
import matplotlib.pyplot as plt

In [11]:
factor_rets = pd.read_excel('data/momentum_data.xlsx', sheet_name='factors (excess returns)').set_index('Date')
MKT_rets = factor_rets['MKT']
SMB_rets = factor_rets['SMB']
HML_rets = factor_rets['HML']

UMD_rets_df = pd.read_excel('data/momentum_data.xlsx', sheet_name='momentum (excess returns)').set_index('Date')
UMD_rets = UMD_rets_df['UMD']

deciles_rets = pd.read_excel('data/momentum_data.xlsx', sheet_name='deciles (total returns)').set_index('Date')
size_sorts_rets = pd.read_excel('data/momentum_data.xlsx', sheet_name='size_sorts (total returns)').set_index('Date')

risk_free_df = pd.read_excel('data/momentum_data.xlsx', sheet_name='risk-free rate').set_index('Date')
r_f = risk_free_df['RF'] 

FREQ = 12  # Monthly data

## 2.1 Is Momentum still profitable?

The excess returns of lucrative trading strategies often disappear once the strategy is well-known. The first widely-cited paper on momentum was published in 1993. Have momentum returns or risk changed since then? The AQR case takes place at the end of 2008. Have momentum returns changed in 2009-2024?

Investigate by filling out the summary statistics below for the full-sample and three sub-samples.

### (a)

Using the data provided, fill in Table 1 with the appropriate stats for $\tilde{r}^{\text{mom,FF}}$.

In [12]:
def calculate_statistics(rets):
    mean = rets.mean() * FREQ
    vol = rets.std() * (FREQ ** 0.5)
    sharpe = mean / vol
    skew = rets.skew()
    corr_to_mkt = rets.corr(MKT_rets.loc[rets.index])
    corr_to_value = rets.corr(HML_rets.loc[rets.index])
    
    stats = {
        'Mean': mean,
        'Volatility': vol,
        'Sharpe Ratio': sharpe,
        'Skewness': skew,
        'Corr to MKT': corr_to_mkt,
        'Corr to HML': corr_to_value
    }
    return stats

In [23]:
results = {}
results['1927 - 2024'] = calculate_statistics(UMD_rets[:'2024-12-31'])
results['1927 - 1993'] = calculate_statistics(UMD_rets.loc[:'1993-12-31'])
results['1993 - 2008'] = calculate_statistics(UMD_rets.loc['1994-01-01':'2008-12-31'])
results['2009 - 2024'] = calculate_statistics(UMD_rets.loc['2009-01-01':'2024-12-31'])
results_df = pd.DataFrame(results).T
display(results_df.style.format("{:.2%}"))


,Mean,Volatility,Sharpe Ratio,Skewness,Corr to MKT,Corr to HML
1927 - 2024,7.39%,16.27%,45.40%,-306.73%,-34.92%,-40.98%
1927 - 1993,8.81%,16.05%,54.91%,-391.10%,-36.75%,-50.10%
1993 - 2008,10.27%,17.54%,58.58%,-59.64%,-24.02%,-11.98%
2009 - 2024,-1.29%,15.80%,-8.16%,-269.16%,-37.02%,-31.90%



### (b)

Has momentum changed much over time, as seen through these subsample statistics?

Until 2008 the strategy performance was pretty stable. The mean returns were pretty strong between 1927 and 2008 (around 8-10%) but has since then been slightly negative since 2009. The volatility has been very similar between the periods and the same can be said of the correlation to the market. However the Skewness and correlation to the value factor have changed slightly, especially in the period from 1994 - 2008 but overall just solid.

### (c)

Does this data support AQR’s argument that momentum is an important piece of the ideal portfolio? What if mean returns to momentum are in actuality near zero due to transaction costs - would there still be evidence here that momentum is valuable?

Yes definitely. The strategy was very profitable when AQR was analysing this product in 2008. The correlation is negative to the market and to value factor so the momentum factor can be a valuable diversifier to ones portfolio. So even if the mean is near zero the factor can increase the investable universe through diversification.